# Updated database functions

`update` removes incidents from the table that have an endtime 5 hours before the current time.

`insert_data` now removes duplicate incidents before appending remaining incidents to the table.

In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
import requests

def update(conn):
    """Updates incidents table by removing incidents with end times five hours before current time"""

    try:
        cutoffTime = datetime.now() + timedelta(hours=24)
        cursor = conn.cursor()
        cmd=\
        f"""
        DELETE FROM testtable
        WHERE endDatetime < '{cutoffTime.strftime('%Y-%m-%d %H:%M:%S')}'
        """
        cursor.execute(cmd)
    except:
        print("Error, likely table does not exist")
        

def insert_data(conn, data):
    """Inserts new traffic data into incidents table in database"""
    
    if len(data) <= 0:
        return
    cursor = conn.cursor()
    # checkcmd = "SHOW TABLES LIKE 'testtable'"
    # cursor.execute(checkcmd)
    # result = cursor.fetchone()
    try:
        data['endDatetime'] = data['endTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
        data.to_sql("intermediate", conn, if_exists='replace', index=False)
        
        # incidents table is replaced here with a test table
        cmd=\
        """
        DELETE FROM intermediate
        WHERE id IN (SELECT id FROM testtable)
        """
        cursor.execute(cmd)
        cursor = conn.cursor()
        cmd=\
        """
        INSERT INTO testtable
        SELECT * FROM intermediate
        """
        cursor.execute(cmd)
        cursor = conn.cursor()
        cmd = "DROP TABLE intermediate"
        cursor.execute(cmd)
    except:
        print('except triggered')
        data['endDatetime'] = data['endTime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))
        data.to_sql("testtable", conn, if_exists="append", index=False)

def get_traffic_data(bbox, key):
    """retrieves traffic incident data in a given bounding box from MapQuest Traffic API"""

    # key = ""
    response = requests.get(f"https://www.mapquestapi.com/traffic/v2/incidents?key={key}&boundingBox={bbox}&filters=congestion,incidents,construction,event")
    data = pd.DataFrame(response.json()["incidents"])
    if len(data) > 0:
        data = data[['id', 'type', 'severity', 'shortDesc', 'lat', 'lng', 'startTime', 'endTime']]
    return data

def get_incidents_in_area(conn, bbox):
    """Retrieves incidents within the given bounding box from database"""
    
    min_lat, max_lat = bbox[0], bbox[1]
    min_lng, max_lng = bbox[2], bbox[3]
    cmd=\
        f"""
            SELECT * FROM testtable
            WHERE lat BETWEEN {min_lat} AND {max_lat}
            AND lng BETWEEN {min_lng} AND {max_lng}
        """
    # print(cmd)
    df = pd.read_sql_query(cmd, conn)
    return df

# Demonstration

In [2]:
import credentials as cred
import requests

key=cred.mapquest_api_key
bbox="40,-122,39,-121"
data = get_traffic_data(bbox, key)

In [3]:
conn = sqlite3.connect('mid_report_test.db')

In [4]:
data

,id,type,severity,shortDesc,lat,lng,startTime,endTime
0,4109331903534083498,1,1,Road maintenance operations,39.19075,-121.05235,2023-05-23T16:01:51,2023-05-24T00:01:51
1,3644140139444008774,1,1,Construction work,39.45720,-121.99664,2023-05-23T13:30:42,2023-05-24T00:01:42
2,608077249452275769,1,1,Construction work,39.51694,-121.01098,2023-02-13T15:01:00,2023-06-02T01:01:00
3,2600782410370498806,1,1,Long-term road construction,39.17099,-121.59002,2022-04-18T07:00:00,2023-05-27T01:59:53
4,220393939384989650,1,2,Road construction,40.03581,-121.00449,2023-05-21T01:47:07,2023-05-23T21:07:07
5,1809922356945229774,1,1,Construction work,39.45736,-121.99356,2023-05-23T13:30:42,2023-05-24T00:01:42
6,4239224218369690062,4,3,Closed,39.79423,-121.45300,2023-05-21T08:04:17,2023-05-24T17:04:17
7,1612766711213318362,1,1,Construction work,39.07734,-121.54057,2023-05-23T14:01:31,2023-05-24T01:01:31
8,908863357339057331,1,1,Construction work,39.50522,-121.02053,2023-02-13T15:01:00,2023-06-02T01:01:00
9,4056656250572595769,1,1,Road maintenance operations,39.36996,-121.10444,2023-05-23T17:01:00,2023-05-24T01:01:00


In [5]:
insert_data(conn, data)

Number of incidents in the table are now 15:

In [6]:
cmd=\
"""
    SELECT * from testtable
"""
df = pd.read_sql_query(cmd, conn)
df

,id,type,severity,shortDesc,lat,lng,startTime,endTime,endDatetime
0,4051814967157092216,1,3,Closed due to road construction,39.79423,-121.45300,2023-05-20T18:46:07,2023-05-24T16:46:07,2023-05-24 16:46:07
1,608077249452275769,1,1,Construction work,39.51694,-121.01098,2023-02-13T15:01:00,2023-06-02T01:01:00,2023-06-02 01:01:00
2,908863357339057331,1,1,Construction work,39.50522,-121.02053,2023-02-13T15:01:00,2023-06-02T01:01:00,2023-06-02 01:01:00
3,1476600228324287985,1,1,Construction work,39.94842,-121.65215,2022-05-05T14:01:00,2023-06-01T00:01:00,2023-06-01 00:01:00
4,2702641940470657655,1,1,Construction work,39.46179,-121.99238,2022-05-05T07:00:00,2023-05-27T01:59:53,2023-05-27 01:59:53
5,2600782410370498806,1,1,Long-term road construction,39.17099,-121.59002,2022-04-18T07:00:00,2023-05-27T01:59:53,2023-05-27 01:59:53
6,4109331903534083498,1,1,Road maintenance operations,39.19075,-121.05235,2023-05-23T16:01:51,2023-05-24T00:01:51,2023-05-24 00:01:51
7,3644140139444008774,1,1,Construction work,39.45720,-121.99664,2023-05-23T13:30:42,2023-05-24T00:01:42,2023-05-24 00:01:42
8,220393939384989650,1,2,Road construction,40.03581,-121.00449,2023-05-21T01:47:07,2023-05-23T21:07:07,2023-05-23 21:07:07
9,1809922356945229774,1,1,Construction work,39.45736,-121.99356,2023-05-23T13:30:42,2023-05-24T00:01:42,2023-05-24 00:01:42


In [7]:
bbox = (38, 41, -123, -120) 
len(get_incidents_in_area(conn, bbox))

15

There are seven incidents that end in less than 24 hours (from the time of this writing).

In [8]:
update(conn)

In [9]:
cmd=\
"""
    SELECT * from testtable
"""
df = pd.read_sql_query(cmd, conn)
df

,id,type,severity,shortDesc,lat,lng,startTime,endTime,endDatetime
0,4051814967157092216,1,3,Closed due to road construction,39.79423,-121.45300,2023-05-20T18:46:07,2023-05-24T16:46:07,2023-05-24 16:46:07
1,608077249452275769,1,1,Construction work,39.51694,-121.01098,2023-02-13T15:01:00,2023-06-02T01:01:00,2023-06-02 01:01:00
2,908863357339057331,1,1,Construction work,39.50522,-121.02053,2023-02-13T15:01:00,2023-06-02T01:01:00,2023-06-02 01:01:00
3,1476600228324287985,1,1,Construction work,39.94842,-121.65215,2022-05-05T14:01:00,2023-06-01T00:01:00,2023-06-01 00:01:00
4,2702641940470657655,1,1,Construction work,39.46179,-121.99238,2022-05-05T07:00:00,2023-05-27T01:59:53,2023-05-27 01:59:53
5,2600782410370498806,1,1,Long-term road construction,39.17099,-121.59002,2022-04-18T07:00:00,2023-05-27T01:59:53,2023-05-27 01:59:53
6,4239224218369690062,4,3,Closed,39.79423,-121.45300,2023-05-21T08:04:17,2023-05-24T17:04:17,2023-05-24 17:04:17


Now we can try updating the same area with an api call:

In [10]:
bbox="40,-122,39,-121"
data = get_traffic_data(bbox, key)
data

,id,type,severity,shortDesc,lat,lng,startTime,endTime
0,608077249452275769,1,1,Construction work,39.51694,-121.01098,2023-02-13T15:01:00,2023-06-02T01:01:00
1,1809922356945229774,1,1,Construction work,39.45736,-121.99356,2023-05-23T13:30:42,2023-05-24T00:01:42
2,3644140139444008774,1,1,Construction work,39.45720,-121.99664,2023-05-23T13:30:42,2023-05-24T00:01:42
3,2702641940470657655,1,1,Construction work,39.46179,-121.99238,2022-05-05T07:00:00,2023-05-27T01:59:53
4,1612766711213318362,1,1,Construction work,39.07734,-121.54057,2023-05-23T14:01:31,2023-05-24T01:01:31
5,627284707635367094,1,1,Road maintenance operations,39.17641,-121.05374,2023-05-23T16:01:41,2023-05-24T00:01:41
6,220393939384989650,1,2,Road construction,40.03581,-121.00449,2023-05-21T01:47:07,2023-05-23T21:07:07
7,2600782410370498806,1,1,Long-term road construction,39.17099,-121.59002,2022-04-18T07:00:00,2023-05-27T01:59:53
8,4056656250572595769,1,1,Road maintenance operations,39.36996,-121.10444,2023-05-23T17:01:00,2023-05-24T01:01:00
9,4239224218369690062,4,3,Closed,39.79423,-121.45300,2023-05-21T08:04:17,2023-05-24T17:04:17


Just by inspection, six of the seven incidents already in our table are repeated in this api call. If we insert this data,  we should have `7 + 14 - 6 = 15` incidents left.

In [11]:
insert_data(conn, data)

In [13]:
cmd=\
"""
    SELECT * from testtable
"""
df = pd.read_sql_query(cmd, conn)
print(f"There are now {len(df)} incidents in the table")

There are now 15 incidents in the table


In [69]:
conn.close()